In [28]:
import pandas as pd
import os
import json
import re
pd.set_option('display.max_colwidth', None)
from examples.arithmetic.increasing_hyp_space import extra_configs
from os.path import join

In [29]:
def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

In [30]:
example = '../../../examples/arithmetic'

In [31]:
def get_mode_declarations():    
    config_results = {}
        
    for c in extra_configs:
        # read config in
        mode_declarations = extra_configs[c].md
        bk = extra_configs[c].bk
        display_md = []
        if 'modeb(var(num) = var(digit_type))' in mode_declarations:
            display_md.append('=')
        if 'even' in mode_declarations:
            display_md.append(r'$\asp{even}$')
        if 'not even' in mode_declarations:
            display_md.append(r'$\asp{not even}$')
        if 'plus_one' in mode_declarations:
            display_md.append('$\asp{plus_nine},...,\asp{plus_one}$')
        elif 'plus_nine' in mode_declarations:
            display_md.append(r'$plus_nine$')

        if '+' in mode_declarations:
            display_md.append('+')
        if ' - ' in mode_declarations:
            display_md.append('-')
        if '*' in mode_declarations:
            display_md.append(r'$\times$')
        if '/' in mode_declarations:
            display_md.append(r'$\div$')


        if 'num(0..18)' in bk:
            label_range = '0..18'
        elif 'num(-9..81)' in bk:
            label_range = '-9..81'

        config_results[c.split('.')[0]] = {
            'mode_declarations': display_md,
            'label_range': label_range
        }
    return config_results

In [32]:
# Generate convergence iteration and run time until convergence automatically
def get_convergence_iteration_and_run_time(example_dir):
    nsl_dir = example_dir+'/saved_results/increasing_hyp_space/e9p/40/'
    configs = os.listdir(nsl_dir)
    configs = [c for c in configs if c != '.DS_Store']
    configs = [c for c in configs if c != '.DS_Store']
    configs = [c for c in configs if c != '__init__']
    configs = [c for c in configs if c != '__init__.py']
    configs = [c for c in configs if c != '__pycache__']
    configs = [c for c in configs if '.json' not in c]
    
    config_results = {}
    for c in configs:
        
        # Read results
        with open(join(nsl_dir+'/'+c, 'test_log.json'), 'r') as jf:
            tl = json.loads(jf.read())
            for e in tl:
                if tl[str(e)]['hyp_accuracy'] >= 1:
                    convergence_epoch = int(e)
                    break
            if convergence_epoch == 0:
                convergence_epoch = 1
        
            # Read times
            train_details_file = nsl_dir+'/'+c+'/train_details.txt'
            iteration_times = []
            read=False
            for line in open(train_details_file):
                if '"total_without_test"' in line.strip():
                    val = line.strip().split(': ')[1]
                    iteration_times.append(float(val))
                    read = True
                    continue
                elif line.strip() == '---------------------':
                    read = False
            # Sum how long it takes to reach this epoch
            epoch_time = 0
            for j in range(int(convergence_epoch)):
                epoch_time += iteration_times[j]
        
            config_results[c] = {"convergence_epoch": convergence_epoch, "time": epoch_time}
    
    return config_results

In [33]:
def create_hyp_space_table(example_dir, caption):
    md = get_mode_declarations()
    iter_run_time = get_convergence_iteration_and_run_time(example_dir)
    
    header = ['Domain Knowledge', 'Label Range', 'Number of Epochs until Convergence', 'Run Time until Convergence (s)']
    data = []
    nsl_dir = example_dir+'/saved_results/increasing_hyp_space/e9p/40'
    configs = os.listdir(nsl_dir)
    configs = [c for c in configs if c != '.DS_Store']
    configs = [c for c in configs if c != '__init__']
    configs = [c for c in configs if c != '__init__.py']
    configs = [c for c in configs if c != '__pycache__']
    configs = [c for c in configs if '.json' not in c]
    configs.sort(key=natural_keys)
    
    for idx, c in enumerate(configs):
        this_md = '. '.join(md[c]['mode_declarations'])
        this_md = this_md.replace('_','\\_')
        this_md = this_md.replace('#','\\#')
        
        lr = md[c]['label_range']
        ce = int(iter_run_time[c]['convergence_epoch'])
        irt = round(iter_run_time[c]['time'],2)
        
        # Bold first result
        if idx == 0:
            this_md = '\textbf{'+this_md+'}'
            lr = r'$\boldsymbol{'+lr+'}$'
            ce = '\textbf{'+str(ce)+'}'
            irt = '\textbf{'+str(irt)+'}'
        else:
            lr = '$'+lr+'$'
            
        data.append([this_md, lr, ce, irt])
    
    df = pd.DataFrame(data, columns=header)
    l= df.to_latex(column_format='M{3cm}M{1.2cm}M{1.8cm}M{2cm}',index=False, escape=False)
    latex_list = l.splitlines()
    for idx,c in enumerate(configs):
        if idx < len(configs)-1:
            latex_list.insert(5+idx*2, '\midrule')
    latex_list.insert(len(latex_list), '\end{table}')
    latex_list.insert(len(latex_list)-1, '\caption{'+caption+'}')
    latex_list.insert(0, '\\begin{table}')
    latex_list.insert(1, '\centering')

    # join split lines to get the modified latex output string
    latex_new = '\n'.join(latex_list)
    
    return latex_new

In [34]:
print(create_hyp_space_table(example, caption="Increasing the hypothesis space, MNIST Alternative Hypothesis task. Bold shows configuration used in this paper."))

\begin{table}
\centering
\begin{tabular}{M{3cm}M{1.2cm}M{1.8cm}M{2cm}}
\toprule
                                                                               Domain Knowledge &          Label Range & Number of Epochs until Convergence & Run Time until Convergence (s) \\
\midrule
                                    \textbf{=. $\asp{even}$. $\asp{not even}$. $plus\_nine$. +} & $\boldsymbol{0..18}$ &                         \textbf{2} &                \textbf{475.44} \\
\midrule
                        =. $\asp{even}$. $\asp{not even}$. $plus\_nine$. +. -. $\times$. $\div$ &              $0..18$ &                                  2 &                         547.81 \\
\midrule
                        =. $\asp{even}$. $\asp{not even}$. $plus\_nine$. +. -. $\times$. $\div$ &             $-9..81$ &                                  2 &                        1725.22 \\
\midrule
                        =. $\asp{even}$. $\asp{not even}$. $sp{plus\_nine},...,sp{plus\_one}$ &              $0..1

/var/folders/lb/jtjzg_ds74q9wpdqvqjlp47w0000gn/T/ipykernel_61144/2823747174.py:37: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  l= df.to_latex(column_format='M{3cm}M{1.2cm}M{1.8cm}M{2cm}',index=False, escape=False)
